## Source Analysis

In [1]:
import duckdb

In [2]:
aws_access_key_id=""
aws_secret_access_key=""
aws_session_token=""
aws_region=""
src_s3_uri="s3://"

In [3]:
create_secret_ddl = f"""
    CREATE OR REPLACE SECRET my_ephem_sts_token 
        ( 
            TYPE S3,
            KEY_ID '{aws_access_key_id}',
            SECRET '{aws_secret_access_key}',
            SESSION_TOKEN '{aws_session_token}',
            REGION '{aws_region}'
        );"""

duckdb.sql(create_secret_ddl).show()

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘



Read in 2 different `parquet` files - one each for `green cabs` and `yellow cabs` for the month of `January, 2024`. Both are available at our internal s3 bucket `source` folder.

Analyse their schema differences and decide on a common table schema. We are going to add both files to the same table, with new additional column,

- `cab_type` e.g. `yellow` or `green`
- `trip_month`
- `trip_year`

We will also treat them as partitions in the table and define some indices on them.

But before all of that can happen, we should check for column parity between both files and ensure we know any transformations we need to do on the files in order to deal with such gaps.

Comparison on only for of the `parquet_schema` fields is enough,
- `name`
- `type`
- `repetition_type` - is the schema field is mandatory or optional?
- `converted_type`

### Green Cab Analysis

In [4]:
green_jan2024_flname=f"{src_s3_uri}/green_tripdata_2024-01.parquet"
read_src_green_sql = f"""
    SELECT
        NAME,
        TYPE,
        REPETITION_TYPE,
        CONVERTED_TYPE
    FROM parquet_schema('{green_jan2024_flname}');
"""
duckdb.sql(read_src_green_sql).show()

┌───────────────────────┬────────────┬─────────────────┬──────────────────┐
│         name          │    type    │ repetition_type │  converted_type  │
│        varchar        │  varchar   │     varchar     │     varchar      │
├───────────────────────┼────────────┼─────────────────┼──────────────────┤
│ schema                │ NULL       │ REQUIRED        │ NULL             │
│ VendorID              │ INT32      │ OPTIONAL        │ NULL             │
│ lpep_pickup_datetime  │ INT64      │ OPTIONAL        │ TIMESTAMP_MICROS │
│ lpep_dropoff_datetime │ INT64      │ OPTIONAL        │ TIMESTAMP_MICROS │
│ store_and_fwd_flag    │ BYTE_ARRAY │ OPTIONAL        │ UTF8             │
│ RatecodeID            │ INT64      │ OPTIONAL        │ NULL             │
│ PULocationID          │ INT32      │ OPTIONAL        │ NULL             │
│ DOLocationID          │ INT32      │ OPTIONAL        │ NULL             │
│ passenger_count       │ INT64      │ OPTIONAL        │ NULL             │
│ trip_dista

### Yellow Cab Analysis

In [5]:
yellow_jan2024_flname=f"{src_s3_uri}/yellow_tripdata_2024-01.parquet"
read_src_yellow_sql = f"""
    SELECT
        NAME,
        TYPE,
        REPETITION_TYPE,
        CONVERTED_TYPE
    FROM parquet_schema('{yellow_jan2024_flname}');
"""
duckdb.sql(read_src_yellow_sql).show()

┌───────────────────────┬────────────┬─────────────────┬──────────────────┐
│         name          │    type    │ repetition_type │  converted_type  │
│        varchar        │  varchar   │     varchar     │     varchar      │
├───────────────────────┼────────────┼─────────────────┼──────────────────┤
│ schema                │ NULL       │ REQUIRED        │ NULL             │
│ VendorID              │ INT32      │ OPTIONAL        │ NULL             │
│ tpep_pickup_datetime  │ INT64      │ OPTIONAL        │ TIMESTAMP_MICROS │
│ tpep_dropoff_datetime │ INT64      │ OPTIONAL        │ TIMESTAMP_MICROS │
│ passenger_count       │ INT64      │ OPTIONAL        │ NULL             │
│ trip_distance         │ DOUBLE     │ OPTIONAL        │ NULL             │
│ RatecodeID            │ INT64      │ OPTIONAL        │ NULL             │
│ store_and_fwd_flag    │ BYTE_ARRAY │ OPTIONAL        │ UTF8             │
│ PULocationID          │ INT32      │ OPTIONAL        │ NULL             │
│ DOLocation

### Summation

- The yellow cabs schema has 20 fields vs 21 fields in green cabs
- `Airport_fee` is only applicable for yellow cabs
- Pickup & drop-off timestamps have differing names, we will rename to just `pickup_datetime` etc.
- `ehail_fee` & `trip_type` columns are applicable for only green cabs


### Final Target Table Schema

This schema shall be used with `AWS Glue Catalog` for now,

Table name - `tbl_nyc_taxi_trips`

Columns,

1. `vendorid`
50. `pickup_datetime`
50. `dropoff_datetime`
50. `ratecodeid`
61. `pickup_location_id`
72. `drop_off_location_id`
83. `passenger_count`
94. `trip_distance`
105. `fare_amount`
116. `extra`
127. `mta_tax`
138. `tip_amount`
149. `tolls_amount`
159. `ehail_fee`
170. `improvement_surcharge`
180. `total_amount`
190. `payment_type`
200. `trip_type`
210. `congestion_surcharge`
211. `airport_fee`
220. `cab_type`
230. `trip_year`
240. `trip_month`

### Green Transform

In [6]:
src_green_sql = f"""
    SELECT
      gr.vendorid,
      gr.lpep_pickup_datetime as pickup_datetime,
      gr.lpep_dropoff_datetime as dropoff_datetime,
      gr.ratecodeid,
      gr.PULocationID as pickup_location_id,
      gr.DOLocationID as drop_off_location_id,
      gr.passenger_count,
      gr.trip_distance,
      gr.fare_amount,
      gr.extra,
      gr.mta_tax,
      gr.tip_amount,
      gr.tolls_amount,
      gr.ehail_fee,
      gr.improvement_surcharge,
      gr.total_amount,
      gr.payment_type,
      gr.trip_type,
      gr.congestion_surcharge,
      0.0 as airport_fee,
      'green' as cab_type,
      '2024' as trip_year,
      'January' as trip_month
    FROM 
      '{src_s3_uri}/green_tripdata_2024-01.parquet' as gr;
"""


jan_green_cab_trips = duckdb.sql(src_green_sql)

jan_green_cab_trips.df()

,VendorID,pickup_datetime,dropoff_datetime,RatecodeID,pickup_location_id,drop_off_location_id,passenger_count,trip_distance,fare_amount,extra,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,cab_type,trip_year,trip_month
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,1.0,236,239,1.0,1.98,12.80,1.0,...,NaN,1.0,21.66,1.0,1.0,2.75,0.0,green,2024,January
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,1.0,65,170,5.0,6.54,30.30,1.0,...,NaN,1.0,42.66,1.0,1.0,2.75,0.0,green,2024,January
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,1.0,74,262,1.0,3.08,19.80,1.0,...,NaN,1.0,28.05,1.0,1.0,2.75,0.0,green,2024,January
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,1.0,74,116,1.0,2.40,14.20,1.0,...,NaN,1.0,16.70,2.0,1.0,0.00,0.0,green,2024,January
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,1.0,74,243,1.0,5.14,22.60,1.0,...,NaN,1.0,31.38,1.0,1.0,0.00,0.0,green,2024,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56546,2,2024-01-31 20:46:00,2024-01-31 20:55:00,NaN,33,25,NaN,0.00,11.58,0.0,...,NaN,1.0,15.72,NaN,NaN,NaN,0.0,green,2024,January
56547,2,2024-01-31 21:06:00,2024-01-31 21:11:00,NaN,72,72,NaN,0.49,11.58,0.0,...,NaN,1.0,12.58,NaN,NaN,NaN,0.0,green,2024,January
56548,2,2024-01-31 21:36:00,2024-01-31 21:40:00,NaN,72,72,NaN,0.52,11.58,0.0,...,NaN,1.0,15.10,NaN,NaN,NaN,0.0,green,2024,January
56549,2,2024-01-31 22:45:00,2024-01-31 22:51:00,NaN,41,42,NaN,1.17,14.22,0.0,...,NaN,1.0,15.22,NaN,NaN,NaN,0.0,green,2024,January


### Yellow Transform

In [7]:
src_yellow_sql = f"""
    SELECT
      yl.vendorid,
      yl.tpep_pickup_datetime as pickup_datetime,
      yl.tpep_dropoff_datetime as dropoff_datetime,
      yl.ratecodeid,
      yl.PULocationID as pickup_location_id,
      yl.DOLocationID as drop_off_location_id,
      yl.passenger_count,
      yl.trip_distance,
      yl.fare_amount,
      yl.extra,
      yl.mta_tax,
      yl.tip_amount,
      yl.tolls_amount,
      0.0 as ehail_fee,
      yl.improvement_surcharge,
      yl.total_amount,
      yl.payment_type,
      0 as trip_type,
      yl.congestion_surcharge,
      yl.airport_fee,
      'yellow' as cab_type,
      '2024' as trip_year,
      'January' as trip_month
    FROM 
      '{src_s3_uri}/yellow_tripdata_2024-01.parquet' as yl;
"""


yellow_cab_trips = duckdb.sql(src_yellow_sql)

yellow_cab_trips.df()

,VendorID,pickup_datetime,dropoff_datetime,RatecodeID,pickup_location_id,drop_off_location_id,passenger_count,trip_distance,fare_amount,extra,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Airport_fee,cab_type,trip_year,trip_month
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,186,79,1.0,1.72,17.70,1.00,...,0.0,1.0,22.70,2,0,2.5,0.0,yellow,2024,January
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,140,236,1.0,1.80,10.00,3.50,...,0.0,1.0,18.75,1,0,2.5,0.0,yellow,2024,January
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,236,79,1.0,4.70,23.30,3.50,...,0.0,1.0,31.30,1,0,2.5,0.0,yellow,2024,January
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,79,211,1.0,1.40,10.00,3.50,...,0.0,1.0,17.00,1,0,2.5,0.0,yellow,2024,January
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,211,148,1.0,0.80,7.90,3.50,...,0.0,1.0,16.10,1,0,2.5,0.0,yellow,2024,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,107,263,NaN,3.18,15.77,0.00,...,0.0,1.0,21.77,0,0,NaN,NaN,yellow,2024,January
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,114,236,NaN,4.00,18.40,1.00,...,0.0,1.0,25.74,0,0,NaN,NaN,yellow,2024,January
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,211,25,NaN,3.33,19.97,0.00,...,0.0,1.0,23.97,0,0,NaN,NaN,yellow,2024,January
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,107,13,NaN,3.06,23.88,0.00,...,0.0,1.0,33.46,0,0,NaN,NaN,yellow,2024,January


### Advanced Schema Design

We have settled at a basic table design, identifying the `fields`, their data types, nullability etc.

We shall now try to achieve a delta table design that is optimized for certain types of queries. This is so because, although the final analytics will be served by duck db with all queries running in-memory, when we try to load data into duck db, the filters we use at that time, will get pushed to the delta lake. An effective design for handling such queries will save us a lot of s3 io.

#### Partition Strategy

- `trip_month`
- `trip_year`
- `cab_type`